# 🧠 AI Pipeline Walkthrough: From Document to Insight

This notebook demonstrates the exact AI workflow used in the **NotebookLLM Clone** application. It runs standalone (without the Django backend) but uses the **same libraries and logic** to show how your data is processed.

### 🚀 The Pipeline
1.  **Ingestion & OCR**: Extract text from PDFs using **Doctr** (Deep Learning OCR).
2.  **Chunking**: Split text into manageable overlapping segments.
3.  **Embeddings**: Convert text to vectors using **SentenceTransformers** (`BAAI/bge-small-en-v1.5`).
4.  **Vector Store**: Index vectors using **FAISS** for fast similarity search.
5.  **Generation**: Retrieve context and answer questions using **Ollama** (`phi3:mini` or configured model).

In [1]:
# 📦 Install Dependencies
# Run this cell if you haven't installed gTTS yet
!pip install gTTS


[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# 📦 Imports & Configuration
import os
import sys
import torch
import numpy as np
import faiss
import requests
import json
from pathlib import Path
from typing import List, Tuple, Dict
from sentence_transformers import SentenceTransformer

# ⚙️ Configuration (Mirroring backend settings)
EMBEDDING_MODEL_NAME = 'BAAI/bge-small-en-v1.5'
OLLAMA_BASE_URL = 'http://localhost:11434'
OLLAMA_MODEL = 'phi3:mini'  # Change if you use a different model in Ollama
CHUNK_SIZE = 512
CHUNK_OVERLAP = 50

# Check for GPU
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"✅ Using Device: {DEVICE.upper()}")

d:\NoteBook LLM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Using Device: CUDA


## 1. Document Processing (OCR)

We use **Doctr** (Document Text Recognition) to extract text. This is a powerful Deep Learning-based OCR engine that handles complex layouts better than standard text extraction.

In [3]:
# Initialize OCR Model (Cached)
ocr_model = None

def get_ocr_model():
    global ocr_model
    if ocr_model is None:
        print("⏳ Loading Doctr OCR model... (this may take a moment)")
        try:
            from doctr.models import ocr_predictor
            ocr_model = ocr_predictor(pretrained=True).to(DEVICE)
            print("✅ OCR Model Loaded!")
        except ImportError:
            print("❌ Doctr not installed. Please install it to use OCR features.")
    return ocr_model

def extract_text_from_pdf(file_path: str) -> str:
    """Extract text from PDF using Doctr (matches backend strategy)"""
    model = get_ocr_model()
    if not model:
        return "[Error: OCR Unavailable]"

    print(f"🔍 Processing {file_path}...")
    from doctr.io import DocumentFile
    
    try:
        # Load PDF
        doc = DocumentFile.from_pdf(file_path)
        # Run Inference
        result = model(doc)
        
        # Reconstruct Text
        full_text = []
        for page_idx, page in enumerate(result.pages):
            page_text = []
            for block in page.blocks:
                for line in block.lines:
                    line_text = " ".join([word.value for word in line.words])
                    page_text.append(line_text)
            
            # Add page marker
            full_text.append(f"--- Page {page_idx + 1} ---\n" + "\n".join(page_text))
        
        return "\n\n".join(full_text)
    except Exception as e:
        return f"❌ Error extracting text: {str(e)}"

## 2. Semantic Chunking

LLMs have context limits. We split the text into overlapping chunks to ensure we can retrieve just the relevant parts.

In [4]:
def chunk_text(text: str, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP) -> List[str]:
    """Split text into overlapping chunks"""
    words = text.split()
    chunks = []
    
    current_chunk = []
    current_len = 0
    
    for word in words:
        word_len = len(word) + 1
        if current_len + word_len > chunk_size and current_chunk:
            chunks.append(" ".join(current_chunk))
            # Overlap: keep last 'overlap' words for next chunk
            current_chunk = current_chunk[-overlap:]
            current_len = sum(len(w) + 1 for w in current_chunk)
        
        current_chunk.append(word)
        current_len += word_len
    
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    print(f"✂️  Created {len(chunks)} chunks from document.")
    return chunks

## 3. Embeddings & Vector Store (RAG Core)

We convert chunks into **Vector Embeddings** (lists of numbers representing meaning) and store them in **FAISS**.

In [5]:
# Load Embedding Model
print(f"⏳ Loading Embedding Model: {EMBEDDING_MODEL_NAME}...")
embedder = SentenceTransformer(EMBEDDING_MODEL_NAME, device=DEVICE)
print("✅ Embedding Model Loaded!")

class VectorStore:
    def __init__(self, dimension=384):
        # Using FAISS IndexFlatIP (Inner Product) for Cosine Similarity
        # (Requires normalized vectors)
        self.index = faiss.IndexFlatIP(dimension)
        self.chunks = [] # Store actual text to retrieve later
    
    def add_documents(self, text_chunks: List[str]):
        # 1. Embed
        embeddings = embedder.encode(text_chunks, convert_to_numpy=True, normalize_embeddings=True)
        
        # 2. Add to Index
        self.index.add(embeddings)
        self.chunks.extend(text_chunks)
        print(f"💾 Stored {len(text_chunks)} vectors in FAISS.")
    
    def search(self, query: str, k=3) -> List[str]:
        # 1. Embed Query
        # (Add instruction prefix if using BGE models)
        query_text = f"Represent this sentence for searching relevant passages: {query}"
        query_embedding = embedder.encode([query_text], convert_to_numpy=True, normalize_embeddings=True)
        
        # 2. Search
        D, I = self.index.search(query_embedding, k)
        
        results = []
        for idx in I[0]:
            if idx >= 0 and idx < len(self.chunks):
                results.append(self.chunks[idx])
        return results

# Initialize Store
vector_store = VectorStore(dimension=embedder.get_sentence_embedding_dimension())

⏳ Loading Embedding Model: BAAI/bge-small-en-v1.5...
✅ Embedding Model Loaded!


## 4. Chat with Ollama

We query your local Ollama instance, providing the retrieved context.

In [6]:
def generate_answer(query: str, context_chunks: List[str]):
    """
    Ask Ollama to answer based ONLY on the context.
    Replicates the exact System Prompt from the backend.
    """
    
    # 1. Construct Context Block
    context_text = "\n\n".join([f"[Source {i+1}]: {chunk}" for i, chunk in enumerate(context_chunks)])
    
    # 2. System Prompt (The "Brain" rules)
    system_prompt = """You are a helpful AI assistant that answers questions based ONLY on the provided source documents.

CRITICAL RULES:
1. Answer based ONLY on the context provided below. NEVER use external knowledge or assumptions.
2. If the information is not explicitly stated in the sources, say "I cannot find this information in the provided documents."
3. When citing information, use the exact source reference format [Source X].
4. Keep answers factual and accurate."""

    # 3. User Prompt
    final_prompt = f"""Context from documents:
{context_text}

---

Question: {query}

Please provide a comprehensive answer based on the sources above."""

    # 4. Call Ollama API
    payload = {
        "model": OLLAMA_MODEL,
        "prompt": final_prompt,
        "system": system_prompt,
        "stream": False,
        "options": {"temperature": 0.7}
    }
    
    try:
        response = requests.post(f"{OLLAMA_BASE_URL}/api/generate", json=payload)
        if response.status_code == 200:
            return response.json()['response']
        else:
            return f"❌ Ollama Error: {response.text}"
    except Exception as e:
        return f"❌ Connection Failed: is Ollama running? {e}"

## 5. Bonus: Summarization & Audio (TTS)

We can also use the model to summarize the document and read the answer aloud using **gTTS** (Google Text-to-Speech).

In [7]:
# NEW: Summarization Function
def summarize_document(context_chunks: List[str]):
    """
    Generate a summary of the document based on the provided chunks.
    """
    # 1. Construct Context Block (Limit to first few chunks to fit context if needed)
    # For this demo, we'll use the first 5 chunks or all of them if less.
    summary_context = "\n\n".join(context_chunks[:5]) 
    
    prompt = f"""Summarize the following document text in a concise and informative paragraph:

{summary_context}

Summary:"""

    payload = {
        "model": OLLAMA_MODEL,
        "prompt": prompt,
        "stream": False,
        "options": {"temperature": 0.5}
    }
    
    try:
        response = requests.post(f"{OLLAMA_BASE_URL}/api/generate", json=payload)
        if response.status_code == 200:
            return response.json()['response']
        else:
            return f"❌ Ollama Error: {response.text}"
    except Exception as e:
        return f"❌ Connection Failed: is Ollama running? {e}"

# NEW: Text-to-Speech Function
def speak_text(text: str):
    """
    Convert text to speech and play it in the notebook.
    """
    try:
        from gTTS import gTTS
        from IPython.display import Audio, display
        import io
        
        print("🔊 Generating Audio...")
        tts = gTTS(text, lang='en')
        
        # Save to a bytes buffer instead of a file
        buffer = io.BytesIO()
        tts.write_to_fp(buffer)
        buffer.seek(0)
        
        # Display Audio widget
        display(Audio(buffer.read(), autoplay=True))
        
    except ImportError:
        print("❌ gTTS not installed. Run `!pip install gTTS` to enable audio.")
    except Exception as e:
        print(f"❌ Audio generation failed: {e}")

## 🧪 Interactive Demo Loop

Run the cell below to upload a file (by path) and chat with it!

In [ ]:
print("🤖 AI RAG Demo Ready!")
print("----------------------")

# 1. Get File
file_path = "./documents/Naveen_Resume.pdf"

if os.path.exists(file_path):
    # 2. Extract
    text = extract_text_from_pdf(file_path)
    print(f"\n📄 Validated Text Extraction ({len(text)} chars)")
    
    # 3. Chunk
    chunks = chunk_text(text)
    
    # 4. Index
    vector_store = VectorStore(dimension=embedder.get_sentence_embedding_dimension())
    vector_store.add_documents(chunks)
    
    print("\n✨ Ready to Chat! Type 'exit' to quit. Type 'summary' to summarize the doc.")
    
    while True:
        query = input("\n❓ Ask a question: ")
        if query.lower() in ['exit', 'quit']:
            break
        
        answer = ""
        
        if query.lower() == 'summary':
             print("\n📝 Generating Summary...")
             answer = summarize_document(chunks)
             print("\n" + answer + "\n")
             print("-" * 50)
             speak_text(answer)
             continue
            
        # 5. Retrieve
        relevant_chunks = vector_store.search(query, k=3)
        print(f"   (Found {len(relevant_chunks)} relevant context chunks)")
        
        # 6. Answer
        print("\n🤖 Generating Answer...")
        answer = generate_answer(query, relevant_chunks)
        print("\n" + answer + "\n")
        print("-" * 50)
        
        # 7. Audio
        speak_text(answer)

else:
    print("❌ File not found. Please check the path.")

🤖 AI RAG Demo Ready!
----------------------
⏳ Loading Doctr OCR model... (this may take a moment)


d:\NoteBook LLM\venv\Lib\site-packages\doctr\models\utils\pytorch.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(archive_path, map_location="cp

✅ OCR Model Loaded!
🔍 Processing ./documents/Naveen_Resume.pdf...

📄 Validated Text Extraction (2722 chars)
✂️  Created 20 chunks from document.
💾 Stored 20 vectors in FAISS.

✨ Ready to Chat! Type 'exit' to quit. Type 'summary' to summarize the doc.
